## 1. 加载并清洗数据
- 只需要跑一次就行

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

In [2]:
# 先将原始文件存储为csv文件，方便处理

df = pd.read_csv('translate0-8000-20000.csv')
print(df.shape) # 查看行列数
df.head() # 前几列数据

(20000, 3)


,Publication number,Title,Abstract
0,BG108387A,Engine Designed On The Principle Of Worm Gear ...,The engine can find an application as a mechan...
1,CN2062130U,Outer Rotary Permanent Magnet A.C. Generator,The utility model provides an outer rotary per...
2,CN206099691U,Permanent Magnet Synchronous Motor,The utility model discloses a permanent magnet...
3,EP1946430A1,Brushless Permanent Magnet Motor/ Generator Wi...,A permanent magnet motor/generator that includ...
4,CN102263444A,With A Permanent Magnet And An Electromagnet T...,The invention discloses a disc type magnetic m...


## 2. 切片并翻译
- 由于谷歌翻译api的访问限制，这一步需要根据实际情况调节range()的大小和范围，少量多次
- 结果输出了从a行到b行，共花了多长时间
- 每一次运行都是在原数据的基础上运行的，后面的输出包括了前面的输出结果
- 为了防止出错，可以适当进行保存并输出

In [ ]:
from google_trans_new import google_translator
from multiprocessing.dummy import Pool as ThreadPool
import time

pool = ThreadPool(30)
detector = google_translator(timeout=10)  
translator = google_translator(timeout=10)
time1 = time.time()
for j in range(8000,8500): # range(a,b),遍历a-b行
    df.iloc[j,1] = translator.translate(df.iloc[j,1], lang_tgt='en') # 翻译Title
    df.iloc[j,2] = translator.translate(df.iloc[j,2], lang_tgt='en') # 翻译Abstract     
pool.close()
pool.join()

time2 = time.time() 
print("Translated, a total of %s s."%(time2 - time1))

# 如果报错了，就缩短a,b的距离，重新跑这一块儿就行

In [37]:
# 保存翻译结果，可以先翻译完，再进行分类的操作，不需要每次都运行，可以在适当环节保存，在运行结束时保存
outputpath='translate0-8000.csv' # 输出路径，本地文件夹路径也可以
df.to_csv(outputpath,sep=',',index=False,header=True)
# 同时，翻译完后，还是应该先看看翻译的结果，有没有乱码，检查稍微手动修改后，再做分类处理